In [3]:
import pickle
import json
import oyaml as yaml
from collections import OrderedDict
from pathlib import Path
import sys
import re
p = Path('.')  / 'cloudmesh' / 'bookmanagerservice' / 'service'
sys.path.append(str(p.absolute()))
from nested_lookup import get_all_keys
import getdata

In [4]:
bks = getdata.getBooks(onlybooks=True)
tstbk = 'Cloud Computing'
bkinfo = getdata.getBooks(onlybooks = False, filename = bks[tstbk])

#All Data
mainData = bkinfo[tstbk]['data']
links = bkinfo[tstbk]['links']
metadata = bkinfo[tstbk]['metadata']
data = pickle.load( open( "returnedData.pkl", "rb" ))

![alt text](sample.jpg "Title")

In [6]:
def rec(data, vals):
    lst2 = []
    for v in vals:
        lst = []
        for d in data:
            if d['parent'] == v:
                lst.append(d['id'])
        if len(lst) > 0:
            g = rec(data,lst)
            lst2.append(({v : g}))
        else:
            lst2.append(v)
    return lst2        

In [7]:
final = {}
for d in data:
    if d['parent'] == '#':
        final[d['id']] = d['children']

In [14]:
temp =  {}
temp2 = {}

for keys, values in final.items():
    lst = rec(data,values)
    temp[keys] = lst
    
    
#Missing PART
#convert names in temp to links
def unpack(content):
    nList = []
    for k, v in content.items():
        for value in v:
            if isinstance(value, dict):
                nList += (unpack(value))
            else:
                nList.append(value)
    return nList

keyList = get_all_keys(temp)
idList = unpack(temp)
temp = str(temp)
for item in idList:
    for i in range(len(mainData)):
        t = mainData[i]
        if item == t['id']:
            temp = temp.replace(item, links[i])
for item in keyList:
    
    if '/' in item:
        
        tempItem = item.split("/")
        temp = temp.replace(item, tempItem[-1])
temp = eval(temp)

#
temp2['metadata'] = metadata
temp2['BOOK'] = [temp]
temp2 = temp2

In [17]:
mystr  = yaml.dump(yaml.load(json.dumps(temp2), Loader=yaml.SafeLoader),default_flow_style = False)
mystr = re.sub(r'([\n])  ([A-Z])', r'\n- \2', mystr)

In [18]:
#print(mystr)
bookTitle = metadata['title']
with open('books/samples/nai_test.yaml', 'w+') as f:
    f.write(mystr)


In [19]:
"""
import hashlib 
hash sample
tst = "this is a test"
result = hashlib.sha256(tst.encode()) 
print("The hexadecimal equivalent of SHA256 is : ") 
print(result.hexdigest()) 
mem = "2e99758548972a8e8822ad47fa1017ff72f06f3ff6a016851f45c398732bc50c"
if mem == result.hexdigest():
    print('success')

"""


'\nimport hashlib \nhash sample\ntst = "this is a test"\nresult = hashlib.sha256(tst.encode()) \nprint("The hexadecimal equivalent of SHA256 is : ") \nprint(result.hexdigest()) \nmem = "2e99758548972a8e8822ad47fa1017ff72f06f3ff6a016851f45c398732bc50c"\nif mem == result.hexdigest():\n    print(\'success\')\n\n'

In [1]:
from generateYAML import yamlGenerator
import pickle

In [2]:
tstbk = 'Cloud Computing'
data = pickle.load( open( "returnedData.pkl", "rb" ))

In [3]:
yamlGenerator(tstbk, data)